# Notities

## Handige links
* [Statline portaal](https://opendata.cbs.nl/statline/portal.html?_la=nl&_catalog=CBS)
* [Snelstartgids open data](https://www.cbs.nl/nl-nl/onze-diensten/open-data/statline-als-open-data/snelstartgids)
* [repo Github voor cbsodata](https://github.com/J535D165/cbsodata)
* [PyPi documentatis cbsodata](https://pypi.org/project/cbsodata/)
* [Sphinx docs cbsodata](https://cbsodata.readthedocs.io/en/latest/index.html)
* [GitHub voorbeelden OData 3](https://github.com/statistiekcbs/CBS-Open-Data-v3)
* [GitHub voorbeelden OData 4](https://github.com/statistiekcbs/CBS-Open-Data-v4/tree/master/Python)
* [Statline voor derden](https://www.cbs.nl/nl-nl/cijfers/statline/statline-voor-derden)
* [CBS open data Statline](https://opendata.cbs.nl/statline/portal.html?_la=nl&_catalog=CBS)
* [PDOK Viewer: Kies bij dataset voor CBS gebiedsindelingen (wijken staan helemaal onderaan) voor kaartweergave](https://www.pdok.nl/viewer/)

# Imports

In [ ]:
import pandas as pd
import cbsodata

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Settings

In [ ]:
GEMEENTES_REGIO_NIJMEGEN = ['Berg en Dal', 'Beuningen', 'Druten', 'Heumen', 'Mook en Middelaar', 'Nijmegen', 'Wijchen']

KEYWORDS = ['wmo', 'gemeente', 'regio', 'postcode', 'zorg', 'werk', 'lonen', 'loon', 'inkomen', 'conjunctuur', 
            'prognose', 'afstand', 'wijk', 'voorziening', 'genees', 'gezond']
URLS = ['opendata.cbs.nl', 'dataderden.cbs.nl']

CBS_OPEN_URL = 'opendata.cbs.nl'
WMO_TABLES = {'2020': '84907NED',
            '2019': '84664NED',
            '2018': '84421NED',
            '2017': '83818NED',
            '2016': '83620NED',
            '2015': '83267NED'}
WIJK_TABLES = {'2020': '84799NED', 
               '2019': '84583NED', 
               '2018': '84286NED', 
               '2017': '83765NED', 
               '2016': '83487NED', 
               '2015': '83220NED', 
               '2014': '82931NED'}

# Functies

In [ ]:
def get_relevant_tables(url:str, 
                        keywords=[], 
                        frequency=['Eenmalig', 'Perjaar', 'Perkwartaal', 'Onregelmatig', 'Permaand', 'Pertweejaar', 'Perhalfjaar', 'Tweemaalperjaar'], 
                        select_columns=['ApiUrl','Identifier', 'ShortTitle','ColumnCount', 'RecordCount', 'Updated', 'Period', 'Summary', 'Frequency', 'ShortDescription', 'ExplanatoryText']):
    """
    Method to get relevant tables in the CBS database based on keywords in the title
    
    :params str url: URL of the catalog of the CBS databases, i.e.: 'opendata.cbs.nl'
    :params list[str] keywords: List of words to search for in the title.
    :params list[str] frequency: List of frequencies (level of accuracy in the data) where tables should be returned.
    :select_columns list[str] select_columns: Columns that should be included in the returned database. 
    """
    tables_list = cbsodata.get_table_list(catalog_url=url)
    df_tables = pd.DataFrame(tables_list)
    print(f"Total number of tables in this url is: {df_tables.shape[0]}")
    print(f"The columns in these tables are: {df_tables.columns}")
    total_identifier_list = []
    for keyword in keywords:
        identifier_list = [i['Identifier'] for i in tables_list if keyword.lower() in i['Title'].lower()]
        total_identifier_list = total_identifier_list + identifier_list
    columns_to_include = list(set(select_columns).intersection(df_tables.columns))
    df = df_tables[(df_tables['Identifier'].isin(total_identifier_list))&(df_tables['Frequency'].isin(frequency))][columns_to_include]
    return df

def get_and_combine_cbs_tables(dict_tables, url='opendata.cbs.nl'):
    """
    Method to get multiple simular tables in the CBS database.
    
    :params dict[str, str] tables: Dictionary with as key the period and as value the table name
    :params str url: URL of the catalog of the CBS databases, i.e.: 'opendata.cbs.nl'
    returns: pd.DataFrame
    """

    print(f"Number of tables to collect: {len(dict_tables)}")
    
    df= pd.DataFrame()
    for interval, table in dict_tables.items():
        print(f"Pythonic iteration {interval} for table {table}")
        try:
            df_sub = pd.DataFrame(cbsodata.get_data(table, catalog_url=url))
            df_sub['interval'] = interval
        except Exception:
            df_sub = pd.DataFrame()
            pass
        df = pd.concat([df, df_sub])
    return df

def rename_and_subset_cols(df, dict_rename, list_cols, include=True):
    
    df = df.rename(columns=dict_rename)
    if include:
        df = df[list_cols]
    else:
        df = df.drop(list_cols, axis=1)
    
    return df

def get_wmo_subtable(df, region=None, period=None, form=None):
    """
    Method to subset the dataframe with WMO data.
    
    :params pd.DataFrame df: DataFrame with the WMO data from CBS with the columns: 
                            [['codering_regio', 'perioden', 'financieringsvorm', 'wmoclienten', 'wmoclientenper1000inwoners']]
    :params str region: String to choose region. Possible strings are: "gemeente", "wijk"
    :params str period: String to choose period. Possible strings are: "jaar", "halfjaar" 
    :params str form: String to choose form of financing. Possible strings are: "totaal", "per type"
    returns: pd.DataFrame
    """
    
    if region == "gemeente":
        df = df[df.codering_regio.str.startswith('GM', na=False)]
    if region == "wijk":
        df = df[df.codering_regio.str.startswith('WK', na=False)]
    if region == "buurt":
        df = df[df.codering_regio.str.startswith('BU', na=False)]
    if period == "jaar":
        df = df[~df.perioden.str.contains("halfjaar", na=False)]
    if period == "halfjaar":
        df = df[df.perioden.str.contains("halfjaar", na=False)]
    if form == "totaal":
        df = df[df.financieringsvorm == "Totaal"]
    if form == "per type":
        df = df[df.financieringsvorm != "Totaal"]
    
    return df

def downcast_variables_dataframe(df):
    # Downcast dataset
    df[df.select_dtypes(include='object').columns] = df.select_dtypes(include='object').astype('category')

    for old, new in [('integer', 'unsigned'), ('float', 'float')]:
        for col in df.select_dtypes(include=old).columns:
            df[col] = pd.to_numeric(df[col], downcast=new)
    return df

# Explore possible tables

In [ ]:
# General table list
toc = pd.DataFrame(cbsodata.get_table_list())
print(f"Shape of all tables: {toc.shape[0]}")
toc.sample(5)

In [ ]:
toc[toc['Identifier'] =='84721NED']

In [ ]:
# URL = 'opendata.cbs.nl'
df_tables_cbs_opendata = get_relevant_tables(url='opendata.cbs.nl', 
                                             keywords=KEYWORDS)
print(f"Shape of all tables: {df_tables_cbs_opendata.shape}")
df_tables_cbs_opendata.sample(5)

In [ ]:
# URL = 'dataderden.cbs.nl'
df_tables_cbs_derden = get_relevant_tables(url='dataderden.cbs.nl', 
                                             keywords=KEYWORDS)
print(f"Shape of all tables: {df_tables_cbs_derden.shape}")
df_tables_cbs_derden.sample(5)

In [ ]:
## Possible code to export possible tables, didn't get that much result

# df_tables_cbs = pd.concat([df_tables_cbs_opendata, df_tables_cbs_derden])
# df_tables_cbs.to_csv('df_tables_cbs.csv')
# df_tables_cbs.shape

# Load data

In [ ]:
%%time
# Get WMO
df_wmo = get_and_combine_cbs_tables(dict_tables=WMO_TABLES, url=CBS_OPEN_URL)
df_wmo_sub = rename_and_subset_cols(df=df_wmo, 
                                    dict_rename={"Codering_3": "codering_regio", "Perioden":"perioden", "Financieringsvorm":"financieringsvorm", "WmoClienten_5":"wmoclienten", "WmoClientenPer1000Inwoners_6":"wmoclientenper1000inwoners"}, 
                                    list_cols=['interval', 'codering_regio', 'perioden', 'financieringsvorm', 'wmoclienten', 'wmoclientenper1000inwoners'])

# df_wmo_total
df_wmo_total = get_wmo_subtable(df=df_wmo_sub, region="wijk", period="jaar", form="totaal")
# df_wmo_total['gemeentenaam'] = df_wmo_total['gemeentenaam'].str.strip()
df_wmo_total = downcast_variables_dataframe(df_wmo_total)
df_wmo_total = df_wmo_total.set_index(['codering_regio', 'perioden'])
df_wmo_total = df_wmo_total.drop(['interval'], axis=1)
df_wmo_total.sample(5)

In [ ]:
%%time
# Get Wijkdata
WIJK_TABLES = {'2020': '84799NED', 
               '2019': '84583NED', 
               '2018': '84286NED', 
               '2017': '83765NED', 
               '2016': '83487NED', 
               '2015': '83220NED', 
               '2014': '82931NED'}

df_wijk = get_and_combine_cbs_tables(dict_tables=WIJK_TABLES, url=CBS_OPEN_URL)
# cols_wijk_stripped = [i.rstrip('0123456789').replace("_", "").lower() for i in list(df_wijk.columns)]
# dict_wijk_cols_renamed = {key: value for key, value in zip(iter(df_wijk.columns), iter(cols_wijk_stripped))}
dict_wijk_cols_renamed = {'ID': 'id',
                          'WijkenEnBuurten': 'wijkenenbuurten',
                          'Gemeentenaam_1': 'gemeentenaam',
                          'SoortRegio_2': 'soortregio',
                          'IndelingswijzigingWijkenEnBuurten_4':'indelingswijzigingwijkenenbuurten',
                          'Codering_3':'codering_regio', 
                          'interval':'perioden'}
df_wijk_sub = rename_and_subset_cols(df=df_wijk, 
                                     dict_rename=dict_wijk_cols_renamed, 
                                     list_cols=['id', 'wijkenenbuurten', 'soortregio', 'indelingswijzigingwijkenenbuurten'], 
                                     include=False)
# df_wijk_sub = df_wijk_sub.rename(columns={'codering':'codering_regio', 'interval':'perioden'})
# df_wijk_sub['codering_regio'] = df_wijk_sub['codering_regio'].apply(lambda x: x.strip())
df_wijk_sub['codering_regio'] = df_wijk_sub['codering_regio'].str.strip()
df_wijk_sub['gemeentenaam'] = df_wijk_sub['gemeentenaam'].str.strip()
df_wijk_total = df_wijk_sub[df_wijk_sub.codering_regio.str.startswith('WK', na=False)]
# df_wijk_total = downcast_variables_dataframe(df_wijk_sub) # krijg value error... snap het niet... 
df_wijk_total = df_wijk_total.set_index(['codering_regio', 'perioden'])
df_wijk_total.sample(5)

In [ ]:
df_dataset_WMO = pd.merge(df_wmo_total, df_wijk_total, how='inner', left_index=True, right_index=True)
df_dataset_WMO.sample(5)

## Write & read dataset

In [ ]:
df_dataset_WMO.to_parquet('../data/df_dataset_WMO.parquet.gzip',
              compression='gzip')

### read dataset (if cbsodata isn't working using cbsodata)

In [ ]:
df_inlezen = pd.read_parquet('../data/df_dataset_WMO.parquet.gzip')
print(f"Shape of dataset: {df_inlezen.shape}")
df_inlezen.sample(5)

## Ways to get a subset of the DataFrame:

In [ ]:
# Subset on columnvalue:
df_dataset_WMO[df_dataset_WMO['gemeentenaam']=='Nijmegen']

# One row / record
df_dataset_WMO.loc[('WK026801', '2019')]

# Multiple rows / records based on combination of the multiindex
df_dataset_WMO.loc[[('WK026801', '2018'), ('WK026802', '2018')]]

# Multiple rows / records for one column (works only for series)
df_dataset_WMO['gemeentenaam'].loc[(['WK026801', 'WK026802'], ['2018', '2019'])]

# Multiple rows based on both indexes:
df_dataset_WMO.loc(axis=0)[['WK026801', 'WK026802'], ['2018', '2019']]

# Subset with IndexSlice
idx = pd.IndexSlice
# Subset on one of the multiindex and select a column
df_dataset_WMO.loc[idx[:, ['2018', '2019']], idx["gemeentenaam"]]
# Subset on one of the multiindex and select all columns
df_dataset_WMO.loc[idx['WK026801', :], idx[:]]

In [ ]:
df_dataset_WMO[df_dataset_WMO['gemeentenaam']=='Nijmegen']

## EDA data

# Klad

In [ ]:
# # DRAAIT LANG ALS JE ALLES OPVRAAGT
# url = 'dataderden.cbs.nl'
# df_to_collect = df_tables_cbs_derden
# dict_cbs_tables = {}
# for i in df_to_collect['Identifier'][:2]:
#     print(i)
#     try:
#         df = pd.DataFrame(cbsodata.get_data(str(i), catalog_url=url))
#     except Exception:
#         df = pd.DataFrame()
#         pass
#     dict_cbs_tables[i] = df
# dict_cbs_tables['40072NED'].sample(10)

In [ ]:
# # Eerste lijst met mogelijke tabellen
# dict_tables = {'regio-indeling 2021': '84929NED',
#                'regio-indeling 2020':'84721NED', 
#                'regio-indeling 2019':'84378NED', 
#                'regio-indeling 2018':'83859NED', 
#                'regio-indeling 2017':'83553NED', 
#                'regio-indeling 2016':'83287NED',
#                 'Bevolking; geslacht, leeftijd, regio': '37259ned',
#                 'Huishoudens; samenstelling, regio': '71486ned',
#                 'Personen; afstand tot ouder, 2014': '83246NED',
#                 'Bevolking; nationaliteit en regio' : '84727NED',
#                 'Bevolking; leeftijd, regio, 2020-2050': '84525NED',
#                 'Bevolkingsontwikkeling, regio, 2020-2050': '84528NED',
#                 'doodsoorzaken':'80202ned',
#                 'Geneesmiddelen; regio (gemeente)': '83251NED',
#                 'DBC-GGZ; personen per diagnose; regio': '84859NED',
#                 'Indicatoren jeugdzorg; gemeenten': '82972NED',
#                 'Jeugdzorgcombinaties': '84136NED',
#                 'Jongeren met jeugdzorg; pers. kenmerken':'84135NED',
#                 'Trajecten jeugdzorg; regio, peildatum': '82968NED',
#                 'Gemeentelijke kosten; jeugdzorg, regio': '83454NED',
#                 'Wmo-cliënten; financiering, wijken, 2020': '84907NED',
#                 'Wmo-cliënten; financiering, wijken, 2019': '84664NED',
#                 'Wmo-cliënten; financiering, wijken, 2018': '84421NED',
#                 'Wmo-cliënten; financiering, wijken, 2017': '83818NED',
#                 'Wmo-cliënten; financiering, wijken 2016': '83620NED',
#                 'Wmo-cliënten; financiering, wijken 2015':'83267NED',
#                 'Wmo_cliënten; type, wijken, 2020': '84908NED',
#                 'Wmo-cliënten; type maatwerk, wijken 2019': '84753NED',
#                 'Wmo-cliënten; type maatwerk, wijken 2018': '84752NED',
#                 'Wmo-cliënten; type maatwerk, wijken 2017': '84751NED',
#                 'Inkomsten eigen bijdr. Wmo-maatwerkvoorz': '84578NED',
#                 'Uitgaven Wmo-maatwerkvoorz.; type, regio': '84580NED',
#                 'Wmo-cliënten die instromen in Wlz 2019': '84812NED',
#                 'Wmo-cliënten die instromen in Wlz 2018': '84599NED',
#                 'Wmo-cliënten die instromen in Wlz 2017': '84579NED',
#                 'Wmo-arrangementen; stand, regio': '83268NED',
#                 'Wmo-cliënten; reden beëindiging, regio': '83266NED',
#                 'Wmo-cliënten; type arrangement, regio':'83262NED',
#                 'Wmo-cliënten; type maatwerk; regio': '84839NED',
#                 'Wmo-gebruik; aantal arrangementen, regio': '83264NED',
#                 'Stapeling Sociaal Domein, 2020': '84906NED',
#                 'Gebruik voorz sociaal dom; wijken, 2019': '84662NED',
#                 'Gebr voorz sociaal domein; wijken, 2018': '84420NED',
#                 'Gebr voorz sociaal domein; wijken, 2017': '83817NED',
#                 'Gebr voorz sociaal domein; wijken 2016': '83619NED',
#                 'Inkomen huishoudens; kenmerken, regio': '84866NED',
#                 'Kerncijfers begrot. heffing per gemeente': '83643NED',
#                 'Gemeenterek.; taakvelden groottekl regio': '84413NED',
#                 'Gem.rekening; balans regio grootteklasse': '71231ned'}

In [ ]:
# # Crasht wegens memory problemen
# df_meta = pd.DataFrame()
# for table in dict_tables.values():
#     df = pd.DataFrame(cbsodata.get_data(table, catalog_url=url))
#     if 'RegioS' in df.columns:
#         regios = df.RegioS.unique()
#     else:
#         regios = "NOTPRESENT"
#     if 'Periode' in df.columns:
#         periode = df.Periode.unique()
#     elif 'Perioden' in df.columns:
#         periode = df.Perioden.unique()
#     else:
#         periode = "NOTPRESENT"
#     df.to_csv(f"{table}.csv")
#     df_meta = df_meta.append({'table':table, 'regios':regios, 'periode':periode}, ignore_index=True)
# df_meta.to_csv('df_meta_long.csv', sep=';')

In [ ]:
# # Get meta data
# table = '84727NED'
# print(cbsodata.get_info(table)['ShortDescription'])
# meta_data = pd.DataFrame(cbsodata.get_meta(table, "DataProperties"))
# meta_data.head(10)

In [ ]:
# # tabel te groot
# table = '84727NED'
# df = pd.DataFrame(cbsodata.get_data(table, catalog_url=url))